<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Machine Learning
### Sommersemester 2023
Prof. Dr. Heiner Giefers

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

#  Principal Component Analysis (PCA)

In diesem Abschnitt betrachten wir ein Verfahren, dass dem unüberwachten Lernen (*unsupervised learning*) zuzuordnen ist, nämlich der Hauptkomponentenzerlegung (engl. *Principal Component Analysis*, PCA).
PCA ein Algorithmus zur Reduzierung der Dimensionalität.
Das bedeutet, dass PCA Linearkombinationen der Merkmale berechnet, die den Datensatz mit möglichtst geringem Informationsverlust beschreiben.
Wir haben die Hauptkomponentenzerlegung bereits im Arbeitsblatt zur Datenvorverarbeitung kennen gelernt und auch verwendet.

Um einen Eindruck von der Funktionsweise der PCA zu bekommen, wollen wir uns einen zufällig generierten Datensatz anschauen:

In [ ]:
np.random.seed(42)
nfeatures = 2
nsize = 200
X = np.dot(np.random.rand(nfeatures, 2), np.random.randn(2, nsize)).T
plt.scatter(X[:, 0], X[:, 1])
plt.xlabel(r"$x$")
plt.ylabel(r"$y$")
plt.axis('equal');

Man sieht in dem Streudiagramm, dass es im vorliegenden Datensatz eine Wechselbeziehung zwischen dem Merkmal *x* und dem Merkmal *y* gibt.
In vorherigen Aufgaben haben eine solche Wechselbeziehung bereits ausgenutzt, z.B. um mittels der Linearen Regression die Variable $y$ durch die Variable *x* zu beschreiben.
Wir haben gesehen, dass man *y* durch ein lineares Modell annähern kann.

Bei der Hauptkomponentenzerlegung wird die Wechselbeziehung zwischen Variablen ebenfalls ausgenutzt, aber nicht mit dem Ziel, ein Merkmal zu beschreiben.
Vielmehr werden die Wechselbeziehungen in Form von Linearkombinantionen der Merkmale (den sog. *Hauptkomponenten*) *gelernt*, um den kompletten Datensatz mit weniger Informationen beschreiben zu können.

Mit *Scikit-Learn* können wir die Hauptkomponenten wie folgt berechnen:

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(X)

Die `fit`-Funktion lernt die Hauptkomponenten des Datensatzes
Im trainierten PCA-Modell sind die Eigenvektoren der Kovarianzmatrix unter dem Attribut `components_` zugreifbar, die Eigenvalues werden als `explained_variance_` abgelegt.

In [ ]:
print(pca.components_)
print(pca.explained_variance_)

Man kann diese Parameter auch leicht selbst berechnen, indem man die Funktionen zur Bestimmung der Kovarianzmatrix (`np.cov`) und der Singulärwertzerlegung (`np.linalg.svd`) von NumPy verwendet:

In [ ]:
kovmat = np.cov(X.T)
u,s,v = np.linalg.svd(kovmat)
print("Eigenvektoren :", v)
print("Eigenwerte:", s)



Die Hauptkomponenten verlaufen in Richtung der Eigenvektoren, die Eigenwerte geben das Maß der Streuung der Datenpunkte entlang dieser Hauptkomponente an.
Im folgenden Diagramm zeigen die Vektorpfeile den Verlauf der Hauptkomponenten an. Die Länge der Pfeile spiegelt die Varianz der Hauptkomponente wieder.

In [ ]:
plt.scatter(X[:, 0], X[:, 1], alpha=0.2)

ursprung = (0,0)
pc = []
colors = ['r', 'b', 'y', 'g']
for i in range(len(v)):
    pc.append((v[i][0]*np.sqrt(s[i]), v[i][1]*np.sqrt(s[i])))
    plt.quiver(*ursprung, *pc[i], color=colors[i], scale=3)



### PCA zur Reduktion der Dimensionalität

Bisher haben das PCA-Verfahren genutzt, um die Hauptkomponenten des Datensatzes zu bestimmen.
Im obigen Beispiel hatten wir zwei Variablen und ebenfalls zwei Hauptkomonenten.
Unser ursprünglicher Datensatz hat sich dadurch nicht verändert.

Um Anzahl der Merkmale reduzieren, können wir aber die Ergebnisse der PCA verwenden undem wir die Werte entlang einer oder mehrerer der kleinsten Hauptkomponenten Null setzen.
Geometrisch gesehen, entspricht dies einer Projektion der Datenpunkte auf die dominierenden Hauptkomponenten.
Mit jedem "weglassen" einer Hauptkomponente verliert man zwar einen Teil der Information aus dem Datensatz (mathematisch gesehen entspricht der Informationsgehalt der *Varianz*). Da es sich aber um die Hauptkomponente mit der Jeweils geringsten Varianz handelt, ins der Informationsverlust minimal.

Betrachten wir nun, was mit unserem obigen Beispiel passiert, wenn wir den Datensatz von zwei auf eine Variable reduzieren:

In [ ]:
pca = PCA(n_components=1)
pca.fit(X)
X_pca = pca.transform(X)
print("Ursprüngliche Größe:   ", X.shape)
print("UTransformierte Größe:", X_pca.shape)

Der transformierte Datensatz ist nur noch halb so groß.
Um zu sehen, wie die Daten projiziert wurden, kann man die inverse Transformation durchführen und die Daten in so wieder auf den ursprünglichen 2-dimensionalen Raum abbilden:

In [ ]:
X_new = pca.inverse_transform(X_pca)
plt.scatter(X[:, 0], X[:, 1], alpha=0.2)
plt.scatter(X_new[:, 0], X_new[:, 1], c='b', alpha=0.8)
plt.axis('equal');
print("Der Datensatz enthält %.1f%c der ursprünglichen Information (Varianz)" % (100*pca.explained_variance_ratio_[0], '%'))

Die hellen Punkte sind die Originaldaten, die dunklen Punkte die transformierten Daten.
Man sieht, dass im transformierten Datensatz alle Punkte entlang einer Geraden laufen, auf die die ursprünglichen Punkte projiziert wurden.

Dieser Datensatz mit reduzierter Dimension ist in gewisser Hinsicht *gut genug*, um die wichtigsten Beziehungen zwischen den Punkten zu erfassen: Trotz der Reduzierung der Dimension und damit der Datenmenge um 50% bleibt die Varianz zu fast 95% erhalten.



### PCA zur Visualisierung

Das Einsatzspektrum von PCA zeigt sich besonders bei Datensätzen mit seer hoher Dimensionen.
Im Folgenden wollen wir die PCA-Methode auf einen Bild-Datensatz mit weit mehr als 2 Merkmalen anwenden:

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
print(f"Ziffern Datensatz mit {digits.data.shape[0]} Datenpunkten und {digits.data.shape[1]} Merkmalen")

_, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 3))
for ax, image, label in zip(axes, digits.images, digits.target):
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    ax.set_title('Training: %i' % label)

Jeder Datenpunkt besteht aus einem grob aufgelösten Graustufenbild von 8-mal-8 Pixeln einer handgeschriebenen Ziffer.
Jedes Pixel wird als Merkmal gedeutet.
In Summe haben wir es hier also mit 64 unterschiedlichen Variablen zu tun.

Wir wollen nun den Datensatz auf zwei Hauptkomponenten reduzieren:

In [ ]:
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(digits.data)
print(digits.data.shape)
print(projected.shape)
print("Der Datensatz enthält noch %.1f%c der ursprünglichen Information (Varianz)"
      % (100*sum(pca.explained_variance_ratio_), '%'))

Da wir nun unsere Ziffern-Bilder durch 2 Merkmale beschreiben, können wir die Datenpunkte in der Ebene plotten:

In [ ]:
plt.scatter(projected[:, 0], projected[:, 1],
            c=digits.target, edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('viridis', 10))
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar();

In [ ]:
pca.explained_variance_ratio_[0], pca.explained_variance_ratio_[1]

Was bedeutet diese VEreinfachung?
Wir sind nun von 64 Pixeln auf 2 Merkmale herunter gekommen, haben unseren Datensatz also um den Faktor 32 verkleinert.
Damit erhalten wir nicht nur weniger Daten, auf eine Modellfunktion lässt sich auf weniger Merkmale hin deutlich leichter trainieren.

Allerdings müssen wir uns die Frage stellen, ob 2 Merkmale hier noch ausreichend für gute Vorhersagen sind.
Wir sehen zwar in der Abbildung, dass die Punktwolken für die einzelnen Ziffern gebündelt auftreten.
Aber die Wolken reichen doch deutlich ineinander, sodass es sehr schwierig sein dürfte, klare Entscheidungsgrenzen zu finden.

Auch die statistischen Kennwerte sprechen dagegen, nur zwei Dimensionen zu verwenden.
Durch die übrig gebliebenen Merkmale, werden nur 28.5% der ursprünglichen Varianz abgedeckt.
Diese Reduktion der Information ist hier vermutlich zu hoch, um einem Klassifizierungsalgorithmus noch eine gute Datenbasis zu liefern.

### Die Anzahl der Komponenten bestimmen

Um eine passende Anzahl von Hauptkomponenten festzulegen, kann uns wieder die allgmeine PCA helfen.
Hierbei werden ja alle Hauptkomponenten gefunden und nach ihrer Varianz absteigend sortiert.
Wir betrachten nun die kummulative, relative Varianz.
Beginnend mit der *wichtigsten* Hauptkomponente summieren wir den Anteil, den die Komponenten an der gesamten Varianz des Datensatzes abbilden, auf.
So entsteht die unten abgebildete Kurve.
Wir sehen, wie viel uns das hinzunehmen einer Weiteren Komponte zum Modell *bringt*.
Um z.B. 90% der Varianz zu erhalten, benötig man ca. 20 Hauptkomponenten:

In [ ]:
pca = PCA().fit(digits.data)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

## PCA zur Rauschunterdrückung

Die HAuptkomponentenzerlegung kann auch verwendet werden, um verrauschte Daten aufzubereiten.
Die Idee ist wie folgt:
Komponenten mit einer Varianz die höher als die des Rauschens ist, sollten durch das Rauschen kaum beeinträchtigt sein.
Wenn wir also nur diese wichtigen Hauptkomponenten werwenden, sollte das Rauschen nach dem wiederherstellen der Daten zu großen Teilen eliminiert sein.

Wir nehmen wieder den Ziffern Datensatz und legen über die Bilder ein künstliches Rauschen:

In [ ]:
def plot_digits(data):
    fig, axes = plt.subplots(4, 10, figsize=(10, 4),
                             subplot_kw={'xticks':[], 'yticks':[]},
                             gridspec_kw=dict(hspace=0.1, wspace=0.1))
    for i, ax in enumerate(axes.flat):
        ax.imshow(data[i].reshape(8, 8),
                  cmap='binary', interpolation='nearest',
                  clim=(0, 16))
plot_digits(digits.data)

Nun fügen wir den Bilddaten ein Gauss'sches Rauschen hinzu:

In [ ]:
np.random.seed(42)
noisy = np.random.normal(digits.data, 4)
plot_digits(noisy)

Man sieht, dass die Bilder nun deutlich *verpixelt* sind.
Auf diesen Daten, berechnen wir nun ein PCA Modell, dass noch 50% der Varianz erhalten soll:

In [ ]:
pca = PCA(0.50).fit(noisy)
pca.n_components_

Um die 50%  der Varianz zu erreichen, müssen 12 Hauptkomponenten verwendet werden.
Wir berechnen nun diese Hauptkomponenten und stellen mit diesen die Bilddaten über eine inverse Transformation wieder her:

In [ ]:
components = pca.transform(noisy)
filtered = pca.inverse_transform(components)
plot_digits(filtered)

Wie man sieht, ist das Ruaschen deutlich unterdrückt.
Die Bilder wirken zwar etwas *weich gezeichnet*, aber die Konturen der ursprünglichen Ziffern sind deutlicher zu erkennen, als bei den verpixelten Bildern.

**Quelle:**

[1] Jake VanderPlas, [*Python Data Science Handbook*](https://jakevdp.github.io/PythonDataScienceHandbook), O'Reilly, 2016